In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import json
# Potential libraries for TLE parsing and orbit calculations:
# from skyfield.api import load, EarthSatellite, wgs84 # Example
# from sgp4.api import Satrec, WGS72              # Example

# --- Configuration ---
# !!! WARNING: Hardcoding credentials is NOT recommended !!!
SPACETRACK_USER = "lichenni@mit.edu" # Use environment variables or other secure method
SPACETRACK_PASSWORD = "elQ!4ITS!Ag!WPRv"  # Use environment variables or other secure method

# !!! REPLACE WITH YOUR 5 ACTUAL STARLINK NORAD IDs !!!
NORAD_IDS = ['55097', '55098', '55099', '55100', '55101'] # Example IDs - REPLACE

# Time periods (use timezone-aware datetimes for comparisons if needed)
STORM_START_DT = datetime(2024, 5, 10, 0, 0, 0, tzinfo=timezone.utc)
STORM_END_DT = datetime(2024, 5, 13, 23, 59, 59, tzinfo=timezone.utc)
QUIET_START_DT = datetime(2024, 5, 1, 0, 0, 0, tzinfo=timezone.utc)
QUIET_END_DT = datetime(2024, 5, 4, 23, 59, 59, tzinfo=timezone.utc)

# Earth constants
MU_KM3_S2 = 3.986004418e5 # Earth gravitational parameter km^3/s^2
EARTH_RADIUS_KM = 6371.0 # Mean Earth radius

# API Details
BASE_URL = "https://www.space-track.org"
LOGIN_URL = f"{BASE_URL}/ajaxauth/login"
QUERY_ENDPOINT = f"{BASE_URL}/basicspacedata/query/class/gp_history"
REQUEST_FORMAT = "json"

# --- Helper Functions ---

# !!! IMPLEMENT THIS using sgp4 or skyfield !!!
def calculate_mean_altitude_from_tle(tle_line1, tle_line2):
    """
    Placeholder: Calculates mean altitude from TLE lines.
    Requires implementation using sgp4 or skyfield library.
    """
    try:
        mean_motion_rev_per_day = float(tle_line2[52:63])
        mean_motion_rad_per_sec = mean_motion_rev_per_day * 2 * np.pi / 86400.0
        if mean_motion_rad_per_sec <= 0: return None # Avoid division by zero or sqrt of negative
        sma_km = (MU_KM3_S2 / mean_motion_rad_per_sec**2)**(1/3)
        mean_altitude = sma_km - EARTH_RADIUS_KM
        return mean_altitude
    except Exception as e:
        # print(f"Debug: Error calculating altitude: {e}")
        return None

def parse_epoch_string(epoch_str):
    """Parses Space-Track EPOCH string to datetime object."""
    try:
        # Handles format like 'YYYY-MM-DD HH:MM:SS' or 'YYYY-MM-DD HH:MM:SS.ffffff'
        # Assume UTC
        return datetime.strptime(epoch_str.split('.')[0], '%Y-%m-%d %H:%M:%S').replace(tzinfo=timezone.utc)
    except ValueError:
        print(f"Warning: Could not parse epoch string: {epoch_str}")
        return None

# --- Main Analysis Function ---
def analyze_decay_rates(session, norad_ids, period_name, start_dt, end_dt):
    """Fetches TLEs, calculates altitudes, and computes decay rates for a list of satellites."""
    print(f"\n--- Analyzing {period_name} Period ({start_dt.date()} to {end_dt.date()}) ---")
    period_decay_rates = []

    # Format dates for Space-Track API query (YYYY-MM-DD)
    start_date_str = start_dt.strftime('%Y-%m-%d')
    end_date_str = end_dt.strftime('%Y-%m-%d') # Query range is inclusive usually

    for sat_id in norad_ids:
        print(f"  Fetching TLEs for {sat_id}...")
        # Construct query parameters for the specific satellite and date range
        # Using EPOCH range query: YYYY-MM-DD--YYYY-MM-DD
        epoch_range = f"{start_date_str}--{end_date_str}"
        query_params = {
            "NORAD_CAT_ID": sat_id,
            "EPOCH": epoch_range,
            "orderby": "EPOCH asc", # Ensure chronological order
            "format": REQUEST_FORMAT,
            "limit": 500 # Limit per request, might need adjustment/pagination if >500 TLEs in period
        }

        try:
            resp_query = session.get(QUERY_ENDPOINT, params=query_params)
            resp_query.raise_for_status()

            if not resp_query.text or resp_query.text.strip() == "[]":
                print(f"  No historical data found for NORAD ID {sat_id} in period {epoch_range}.")
                continue

            tle_data = resp_query.json() # Parse JSON response

            altitudes = []
            timestamps_seconds = []
            valid_tle_count = 0
            for record in tle_data:
                # Check if required fields exist
                if "EPOCH" in record and "TLE_LINE1" in record and "TLE_LINE2" in record:
                    epoch_dt = parse_epoch_string(record["EPOCH"])
                    if epoch_dt is None: continue # Skip if date parsing failed

                    # Ensure TLE is within the precise datetime range (API filter is by day)
                    if start_dt <= epoch_dt <= end_dt:
                        alt = calculate_mean_altitude_from_tle(record["TLE_LINE1"], record["TLE_LINE2"])
                        if alt is not None:
                            altitudes.append(alt)
                            timestamps_seconds.append(epoch_dt.timestamp())
                            valid_tle_count += 1

            print(f"  Found {len(tle_data)} records, processed {valid_tle_count} valid TLEs for {sat_id}.")

            if len(altitudes) < 2:
                print(f"  Skipping {sat_id}: Insufficient valid TLEs/altitudes ({len(altitudes)}) for decay calculation.")
                continue

            # Convert timestamps to days relative to the first timestamp for stable fitting
            timestamps_days = (np.array(timestamps_seconds) - timestamps_seconds[0]) / (24.0 * 3600.0)

            # Fit a linear polynomial (degree 1)
            coeffs = np.polyfit(timestamps_days, altitudes, 1)
            decay_rate_km_per_day = coeffs[0]
            period_decay_rates.append(decay_rate_km_per_day)
            print(f"  Decay rate for {sat_id}: {decay_rate_km_per_day:.3f} km/day")

        except requests.exceptions.HTTPError as e:
            print(f"  HTTP Error fetching data for {sat_id}: {e.response.status_code}. Skipping.")
            print(f"  Response: {e.response.text[:200]}...") # Print beginning of error
        except requests.exceptions.RequestException as e:
            print(f"  Network Error fetching data for {sat_id}: {e}. Skipping.")
        except json.JSONDecodeError as e:
            print(f"  Error parsing JSON response for {sat_id}: {e}. Skipping.")
        except Exception as e:
             print(f"  Unexpected error processing {sat_id}: {e}. Skipping.")


    # Calculate average decay rate for the period
    if period_decay_rates:
        avg_decay_rate = np.mean(period_decay_rates)
        print(f"  Average {period_name} decay rate: {avg_decay_rate:.3f} km/day")
        return avg_decay_rate
    else:
        print(f"  Could not calculate average decay rate for {period_name} period.")
        return np.nan

# --- Main Execution ---
session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0"}) # Basic User-Agent
quiet_avg_decay = np.nan
storm_avg_decay = np.nan

try:
    # Login
    login_data = {"identity": SPACETRACK_USER, "password": SPACETRACK_PASSWORD}
    print("Attempting Space-Track login...")
    resp_login = session.post(LOGIN_URL, data=login_data)
    resp_login.raise_for_status()
    # Basic check for successful login (Space-Track usually returns non-HTML on success)
    if "html" in resp_login.headers.get("Content-Type", "").lower():
         raise Exception("Login Failed: Received HTML content instead of expected response.")
    print("Login successful.")

    # Analyze Quiet Period
    quiet_avg_decay = analyze_decay_rates(session, NORAD_IDS, 'Quiet', QUIET_START_DT, QUIET_END_DT)

    # Analyze Storm Period
    storm_avg_decay = analyze_decay_rates(session, NORAD_IDS, 'Storm', STORM_START_DT, STORM_END_DT)

    # Compare Results
    print("\n--- Comparison ---")
    print(f"Average Quiet Period Decay Rate: {quiet_avg_decay:.3f} km/day")
    print(f"Average Storm Period Decay Rate: {storm_avg_decay:.3f} km/day")

    if not np.isnan(quiet_avg_decay) and not np.isnan(storm_avg_decay):
        difference = storm_avg_decay - quiet_avg_decay
        print(f"Difference (Storm - Quiet): {difference:.3f} km/day")
        if storm_avg_decay < quiet_avg_decay:
            print(f"Altitude decay was faster during the storm by {abs(difference):.3f} km/day on average.")
        else:
            print("Altitude decay was not significantly faster during the storm based on this sample.")
        if quiet_avg_decay != 0:
             ratio = storm_avg_decay / quiet_avg_decay
             print(f"Ratio (Storm / Quiet): {ratio:.2f}")
    else:
        print("Could not compute comparison due to missing results for one or both periods.")


except requests.exceptions.Timeout as e:
    print(f"ERROR: Request timed out: {e}")
except requests.exceptions.HTTPError as e:
    print(f"ERROR: HTTP Error: {e.response.status_code} {e.response.reason}")
    print(f"Response text: {e.response.text}")
except requests.exceptions.RequestException as e:
    print(f"ERROR: A network request error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    if session:
        session.close()
        print("Session closed.")

Attempting Space-Track login...
Login successful.

--- Analyzing Quiet Period (2024-05-01 to 2024-05-04) ---
  Fetching TLEs for 55097...
  Found 1 records, processed 0 valid TLEs for 55097.
  Skipping 55097: Insufficient valid TLEs/altitudes (0) for decay calculation.
  Fetching TLEs for 55098...
  Found 1 records, processed 0 valid TLEs for 55098.
  Skipping 55098: Insufficient valid TLEs/altitudes (0) for decay calculation.
  Fetching TLEs for 55099...
  Found 1 records, processed 0 valid TLEs for 55099.
  Skipping 55099: Insufficient valid TLEs/altitudes (0) for decay calculation.
  Fetching TLEs for 55100...
  Found 1 records, processed 0 valid TLEs for 55100.
  Skipping 55100: Insufficient valid TLEs/altitudes (0) for decay calculation.
  Fetching TLEs for 55101...
  Found 1 records, processed 0 valid TLEs for 55101.
  Skipping 55101: Insufficient valid TLEs/altitudes (0) for decay calculation.
  Could not calculate average decay rate for Quiet period.

--- Analyzing Storm Period